In [ ]:
from simple_mamba import all_at_the_same_time, K_regression, alpha_regression, segmentation_model
from Dataloader import list_directory_tree, DatasetAllData, apply_padding
import torch

In [ ]:
model_seg_reg = torch.load("more_features_50epoch_10kfiles")
model_seg_reg.eval()

In [ ]:
data_path = "track_2/"
directory_files = list_directory_tree(data_path)

In [ ]:
def read_dataset(file):
    df = pd.read_csv(file)
    nb_trajs = len(df["traj_idx"].unique())

    df_padded = apply_padding(df)
    df_padded = df_padded[:, :, :]


    return torch.from_numpy(df_padded.astype(np.float32))

In [ ]:
results_alpha = {}
results_K = {}
results_segmentation = {}

with torch.no_grad():
    for file in directory_files:
        data = read_dataset(file)
        data = data.to("cuda")

        segmentation, alpha, K = model_seg_reg(data)
        results_alpha[file] = alpha.squeeze()
        results_K[file] = K.squeeze()
        results_segmentation[file] = torch.argmax(segmentation.squeeze(), dim=-1)


In [ ]:
for i in keys:
    K = results_K[i].to("cpu")
    plt.plot(K[:, 0])

In [ ]:
j = 0
for i in keys:
    K = results_K[i].to("cpu")
    plt.plot(K[:, 1])
    j += len(K[:, 1])

In [ ]:
liste_unique = []
plt.figure()
for i in keys[:1]:

    segmentation = results_segmentation[i].to("cpu")
    # segmentation[segmentation==0] = 1
    plt.plot(segmentation)
    # a = len(np.unique(results_segmentation))
    # liste_unique.append(a)

# plt.plot(liste_unique)

In [ ]:
## As a post-treatment task, we need to remove the segments which size are smaller than 3 as not possible

In [ ]:
plt.plot(results_segmentation[keys[-4]][19,:].to("cpu"))

In [ ]:
def change_small_sections(arr):
    # Find the start and end indices of sections
    diff = np.diff(arr)
    section_starts = np.where(diff != 0)[0] + 1
    section_starts = np.concatenate(([0], section_starts))
    section_ends = np.concatenate((section_starts[1:], [len(arr)]))
    # Iterate through sections
    for start, end in zip(section_starts, section_ends):
        section_length = end - start
        if section_length < 3:

            arr[start:end] =  3 - arr[start:end] 

    return arr

for i in keys:
        for j in range(results_segmentation[i].size(0)):
            
            results_segmentation[i][j,:] = change_small_sections(results_segmentation[i][j,:].to("cpu"))
                


In [ ]:
liste_unique = []
plt.figure()
for i in keys[:1]:

    segmentation = results_segmentation[i].to("cpu")
    # segmentation[segmentation==0] = 1
    plt.plot(segmentation)
    # a = len(np.unique(results_segmentation))
    # liste_unique.append(a)

# plt.plot(liste_unique)

In [ ]:
## Remove all the samm sections



In [ ]:
def generate_solution(file):
    
    df = pd.read_csv(file)

    total_sol = ""
    for id, group in df.groupby("traj_idx"):
        sol=""

        states = results_segmentation[file][int(id)].to("cpu") - 1
        alphas = results_alpha[file][int(id)].to("cpu")
        Ds = results_K[file][int(id)].to("cpu")
        unique_states = np.unique(states)
        
        if len(unique_states) == 1:
            
            if alphas[0] < 0.01:
                state = 1
            elif alphas[0] > 1.8:
                state = 3
            else:
                state = 2

            total_sol+= f"{int(id)},{Ds[0]},{alphas[0]},{state},{len(group["x"])}\n"
        else:

            state = [0,0]
            if alphas[0] < 0.001:
                state[0] = 1
            elif alphas[0] > 1.8:
                state[0] = 3
            else:
                state[0] = 2

            if alphas[1] < 0.001:
                state[1] = 1
            elif alphas[1] > 1.8:
                state[1] = 3
            else:
                state[1] = 2

            if Ds[0] < 1e-7:
                Ds[0] = 0
            if Ds[1] < 1e-7:
                Ds[1] = 0


            init_state = int(states[0])
            sol = f"{int(id)},{Ds[init_state]},{alphas[init_state]},{state[init_state]},"

            for i in range(len(group["frame"]) -1):
                if int(states[i]) != init_state:
                    init_state = (init_state + 1) % 2
                    sol += f"{int(group["frame"].iloc[i])},{Ds[init_state]},{alphas[init_state]},{state[init_state]},"
            sol+=f"{len(group["x"])}\n"
            total_sol+=sol

    return total_sol

In [ ]:
dict_solutions = {}
j=0
for i in keys:
    j=0
    dict_solutions[i] = generate_solution(i)

In [ ]:
keys = list(dict_solutions.keys())

In [ ]:
print(generate_solution(keys[5]))

In [ ]:
from copy import copy
from pathlib import Path

In [ ]:
for i in keys:
    key = copy(i)
    file_name = i.parts[-1][6:-4] + ".txt"
    total_path = Path(*["solution", *i.parts[-3:-1],file_name])
    folder_path = Path(*total_path.parts[:-1])
    folder_path.mkdir(parents=True, exist_ok=True)

    with open(total_path, "w") as file:
        file.write(dict_solutions[key])
